<a href="https://colab.research.google.com/github/MehraeenTimas/nlp-course/blob/main/quiz_2_fakeReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fake Reviews Detection**

In [47]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [18]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**1-extracting data**

In [21]:
#df = pd.read_csv('fake.csv')

In [48]:

df = pd.read_csv('fake.csv',
                 engine='python',
                 on_bad_lines='skip',  # Skip malformed rows
                 quoting=3
                 )            # Ignore quotes (common in messy review CSVs)

print(df.head())
print(f"Loaded {len(df)} rows successfully.")

                       category  \
Home_and_Kitchen_5 5.0       CG   
                   5.0       CG   
                   5.0       CG   
                   1.0       CG   
                   5.0       CG   

                                                                   rating  \
Home_and_Kitchen_5 5.0                             "Love this!  Well made   
                   5.0                                           "love it   
                   5.0  This pillow saved my back. I love the look and...   
                   1.0              "Missing information on how to use it   
                   5.0  Very nice set. Good quality. We have had the s...   

                                                                    label  \
Home_and_Kitchen_5 5.0                                             sturdy   
                   5.0   a great upgrade from the original.  I've had ...   
                   5.0                                               None   
                  

In [52]:
df = df.dropna()

**2-preprocessing**

In [53]:
# Define preprocessing function
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
punct = string.punctuation

def preprocess_text(text):
    if pd.isna(text):
        return ""
    # 1. Lowercase
    text = text.lower()
    # 5. Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # 6. Tokenize, remove stopwords, lemmatize
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply to 'text_' column
df['cleaned_text'] = df['text_'].apply(preprocess_text)

# Optional: drop rows with empty cleaned text
df = df[df['cleaned_text'].str.strip() != '']

print(df[['text_', 'cleaned_text']].head())

                                                                    text_  \
Home_and_Kitchen_5 5.0      and very comfortable.  I love it!Very pretty"   
                   5.0                                    the handle has"   
                   5.0                        and the quality is decent."   
                   1.0   cooked evenly. The only problem is that it's ...   
                   5.0   and easy to clean. I will be purchasing more!...   

                                                    cleaned_text  
Home_and_Kitchen_5 5.0         comfortable. love it!very pretty"  
                   5.0                               handle has"  
                   5.0                          quality decent."  
                   1.0          cooked evenly. problem really a"  
                   5.0  easy clean. purchasing more!very pretty"  


**train test**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt

In [56]:

X = df['cleaned_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train size: {len(X_train)}")
print(f"Test size:  {len(X_test)}")
print(f"Class distribution in train:\n{y_train.value_counts(normalize=True)}")
print(f"Class distribution in test:\n{y_test.value_counts(normalize=True)}")

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X = df['cleaned_text']
y = df['label']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)


In [63]:
print(f"Train size: {len(X_train)}")
print(f"Test size:  {len(X_test)}")
print(f"Class distribution in train:\n{y_train.value_counts(normalize=True)}")
print(f"Class distribution in test:\n{y_test.value_counts(normalize=True)}")

NameError: name 'X_train' is not defined

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
if 'Classifier' in str(type(model)):
    score = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {score:.4f}")
else:
    score = mean_squared_error(y_test, y_pred)
    print(f"MSE: {score:.4f}")


In [ ]:
xgb.plot_importance(model, max_num_features=10)
plt.title("Top Features - XGBoost")
plt.show()